In [9]:
import openfermion as of
from openfermion import FermionOperator
from openfermion.transforms import normal_ordered
from pyblock2._pyscf.ao2mo import integrals as itg

In [10]:
from pyscf import gto, scf

mol = gto.M(
    atom=f"N 0 0 0; N 0 0 2.2",
    basis='sto-3g')
mf = scf.RHF(mol).run(conv_tol=1E-14)
mycc = mf.CISD().run()
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_rhf_integrals(mf,
                                                                     ncore=0,
                                                                     g2e_symm=1)


converged SCF energy = -106.751831266188
E(RCISD) = -107.2227326317978  E_corr = -0.4709013656102005


In [11]:
def construct_one_body_fermion_operator(ferm_op, obt):

    num_orbitals = obt.shape[0]  # assuming h1e is square
    num_spin_orbs = 2 * num_orbitals
    for p in range(num_orbitals):
        for q in range(num_orbitals):
            coefficient = obt[p, q]
            if coefficient != 0.0:
                
                term_alpha_alpha = FermionOperator(((2*p, 1), (2*q, 0)), coefficient)
                ferm_op += term_alpha_alpha
    
                # Beta-beta: (2*p+1, 1) and (2*q+1, 0)
                term_beta_beta = FermionOperator(((2*p+1, 1), (2*q+1, 0)), coefficient)
                ferm_op += term_beta_beta
    
    return ferm_op


def construct_two_body_fermion_operator(ferm_op, tbt):

    num_orbitals = tbt.shape[0]  # assuming h1e is square
    for p in range(num_orbitals):
        for q in range(num_orbitals):
            for r in range(num_orbitals):
                for s in range(num_orbitals):
                    coefficient = tbt[p, q, r, s]
                    if coefficient != 0.0:
                        term_1 = FermionOperator(((2*p, 1), (2*q, 0), (2*r, 1), (2*s, 0)), coefficient)
                        term_2 = FermionOperator(((2*p + 1, 1), (2*q+1, 0), (2*r, 1), (2*s, 0)), coefficient)
                        term_3 = FermionOperator(((2*p, 1), (2*q, 0), (2*r + 1, 1), (2*s + 1, 0)), coefficient)
                        term_4 = FermionOperator(((2*p+1, 1), (2*q+1, 0), (2*r + 1, 1), (2*s + 1, 0)), coefficient)
                        
                        ferm_op += term_1 + term_2 + term_3 + term_4
    
    return ferm_op

In [12]:
def physicist_to_chemist(obt, tbt):
    n = obt.shape[0]
    obt_phy = obt.copy()
    for p in range(n):
        for q in range(n):
            obt_phy[p, q] -= 0.5 * sum([tbt.copy()[p, r, r, q] for r in range(n)])

    return obt_phy, 0.5 * tbt.copy()

In [14]:
from pyscf import gto, scf
from pyblock2._pyscf.ao2mo import integrals as itg

mol = gto.M(atom="O 0 0 0; H 0 1 0; H 0 0 1", basis="sto3g", verbose=0)
mf = scf.RHF(mol).run(conv_tol=1E-14)
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_rhf_integrals(mf,
    ncore=0, ncas=None, g2e_symm=1)

In [18]:
print(ncas, n_elec)

7 10


In [15]:
obt, tbt = physicist_to_chemist(h1e, g2e)
fermioOp = FermionOperator()
fermioOp = construct_one_body_fermion_operator(fermioOp, obt)
fermioOp = construct_two_body_fermion_operator(fermioOp, tbt)

In [16]:
import pickle
with open(f'H2O_sto-3g.pkl', 'wb') as f:
    pickle.dump(fermioOp, f)

In [17]:
print(fermioOp)

-35.11541826381325 [0^ 0] +
2.3727357887716045 [0^ 0 0^ 0] +
-0.20880071880916634 [0^ 0 0^ 2] +
-0.0971208262450535 [0^ 0 0^ 6] +
-0.10744932651419747 [0^ 0 0^ 10] +
2.3727357887716045 [0^ 0 1^ 1] +
-0.20880071880916634 [0^ 0 1^ 3] +
-0.0971208262450535 [0^ 0 1^ 7] +
-0.10744932651419747 [0^ 0 1^ 11] +
-0.20880071880916634 [0^ 0 2^ 0] +
0.5026787523479976 [0^ 0 2^ 2] +
0.07354112219970899 [0^ 0 2^ 6] +
0.14728160556535197 [0^ 0 2^ 10] +
-0.20880071880916634 [0^ 0 3^ 1] +
0.5026787523479976 [0^ 0 3^ 3] +
0.07354112219970899 [0^ 0 3^ 7] +
0.14728160556535197 [0^ 0 3^ 11] +
0.40237871561234906 [0^ 0 4^ 4] +
0.17971283726775464 [0^ 0 4^ 12] +
0.40237871561234906 [0^ 0 5^ 5] +
0.17971283726775464 [0^ 0 5^ 13] +
-0.0971208262450535 [0^ 0 6^ 0] +
0.07354112219970899 [0^ 0 6^ 2] +
0.48027566047176085 [0^ 0 6^ 6] +
-0.12286786286050973 [0^ 0 6^ 10] +
-0.0971208262450535 [0^ 0 7^ 1] +
0.07354112219970899 [0^ 0 7^ 3] +
0.48027566047176085 [0^ 0 7^ 7] +
-0.12286786286050973 [0^ 0 7^ 11] +
0.557672

In [25]:
Hamil_test = FermionOperator('5^ 6^ 8 7') + FermionOperator('7^ 8^ 6 5')

In [26]:
fermT = FermionOperator('1^ 2^ 4 3') - FermionOperator('3^ 4^ 2 1')

In [27]:
print(fermT)

1.0 [1^ 2^ 4 3] +
-1.0 [3^ 4^ 2 1]


In [29]:
commut = Hamil_test * fermT - fermT * Hamil_test
print(commut)

-1.0 [1^ 2^ 4 3 5^ 6^ 8 7] +
-1.0 [1^ 2^ 4 3 7^ 8^ 6 5] +
1.0 [3^ 4^ 2 1 5^ 6^ 8 7] +
1.0 [3^ 4^ 2 1 7^ 8^ 6 5] +
1.0 [5^ 6^ 8 7 1^ 2^ 4 3] +
-1.0 [5^ 6^ 8 7 3^ 4^ 2 1] +
1.0 [7^ 8^ 6 5 1^ 2^ 4 3] +
-1.0 [7^ 8^ 6 5 3^ 4^ 2 1]


In [30]:
print(normal_ordered(commut))

0


In [31]:
commut = fermioOp * fermT - fermT * fermioOp

In [32]:
print(normal_ordered(commut))

-0.15738195542525085 [2^ 1^ 0^ 4 2 1] +
-0.0017410356884446088 [2^ 1^ 0^ 4 3 0] +
-0.09800101685114543 [2^ 1^ 0^ 5 4 2] +
-0.09767291210345871 [2^ 1^ 0^ 6 3 2] +
0.04308407232023098 [2^ 1^ 0^ 6 4 1] +
0.05296046723708304 [2^ 1^ 0^ 6 5 4] +
-0.08424764189095392 [2^ 1^ 0^ 7 4 0] +
0.19600203370229086 [2^ 1^ 2 1] +
-0.16976107153396663 [2^ 1^ 3 0] +
0.6011393633401498 [2^ 1^ 4 3] +
-0.1372828399321162 [2^ 1^ 5 2] +
-0.15063337934054175 [2^ 1^ 6 1] +
-0.09936654029346029 [2^ 1^ 6 5] +
0.09851292568654066 [2^ 1^ 7 0] +
0.12665322153403788 [2^ 1^ 7 4] +
-0.16976107153396675 [3^ 0^ 2 1] +
0.15738195542525085 [3^ 0^ 4 3] +
0.15738195542525085 [3^ 1^ 0^ 4 3 1] +
0.10783206349487381 [3^ 1^ 0^ 5 2 1] +
0.09800101685114543 [3^ 1^ 0^ 5 4 3] +
0.15738195542525085 [3^ 2^ 0^ 4 3 2] +
-0.045552458449457614 [3^ 2^ 0^ 6 2 1] +
0.04308407232023098 [3^ 2^ 0^ 6 4 3] +
-0.19600203370229086 [3^ 2^ 1^ 3 2 1] +
-0.1372828399321162 [3^ 2^ 1^ 5 3 2] +
-0.15063337934054175 [3^ 2^ 1^ 6 3 1] +
0.09936654029346029 [3